In [64]:
# Dependencies and Setup
import hvplot.pandas
import holoviews


# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import json
import datetime

# Import citipy to determine the cities based on latitude and longitude
from citipy import citipy

# Import API key
from api_keys import geoapify_key


ModuleNotFoundError: No module named 'hvplot.points'

In [8]:
hap_df = pd.read_csv('./Resources/2022.csv')

cost_df = pd.read_csv('./Resources/Cost_of_Living_Index_2022.csv')

print(hap_df.head())
print(cost_df.head())

   RANK      Country Happiness score Whisker-high Whisker-low  \
0     1      Finland           7,821        7,886       7,756   
1     2      Denmark           7,636        7,710       7,563   
2     3      Iceland           7,557        7,651       7,464   
3     4  Switzerland           7,512        7,586       7,437   
4     5  Netherlands           7,415        7,471       7,359   

  Dystopia (1.83) + residual Explained by: GDP per capita  \
0                      2,518                        1,892   
1                      2,226                        1,953   
2                      2,320                        1,936   
3                      2,153                        2,026   
4                      2,137                        1,945   

  Explained by: Social support Explained by: Healthy life expectancy  \
0                        1,258                                 0,775   
1                        1,243                                 0,777   
2                        1

In [9]:
# Merging
mHapCost_df = hap_df.merge(cost_df, how='inner', on='Country')
mHapCost_df.head()

,RANK,Country,Happiness score,Whisker-high,Whisker-low,Dystopia (1.83) + residual,Explained by: GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,Explained by: Generosity,Explained by: Perceptions of corruption,Rank,Cost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index
0,1,Finland,"7,821","7,886","7,756","2,518","1,892","1,258","0,775","0,736","0,109","0,534",41,73.20,25.95,51.05,65.16,77.65,91.02
1,2,Denmark,"7,636","7,710","7,563","2,226","1,953","1,243","0,777","0,719","0,188","0,532",33,84.12,33.23,60.26,68.60,98.75,99.45
2,3,Iceland,"7,557","7,651","7,464","2,320","1,936","1,320","0,803","0,718","0,270","0,191",52,94.86,41.93,70.05,90.22,99.42,77.06
3,4,Switzerland,"7,512","7,586","7,437","2,153","2,026","1,226","0,822","0,677","0,147","0,461",120,123.35,53.54,90.62,128.13,122.09,118.44
4,5,Netherlands,"7,415","7,471","7,359","2,137","1,945","1,206","0,787","0,651","0,271","0,419",86,75.66,36.06,57.10,65.49,76.35,87.99


In [10]:
mHapCost_df.count()

RANK                                          106
Country                                       106
Happiness score                               106
Whisker-high                                  106
Whisker-low                                   106
Dystopia (1.83) + residual                    106
Explained by: GDP per capita                  106
Explained by: Social support                  106
Explained by: Healthy life expectancy         106
Explained by: Freedom to make life choices    106
Explained by: Generosity                      106
Explained by: Perceptions of corruption       106
Rank                                          106
Cost of Living Index                          106
Rent Index                                    106
Cost of Living Plus Rent Index                106
Groceries Index                               106
Restaurant Price Index                        106
Local Purchasing Power Index                  106
dtype: int64

In [56]:
top20Hap = mHapCost_df.sort_values('Happiness score', ascending=False).head(20)
top20Hap

,RANK,Country,Happiness score,Whisker-high,Whisker-low,Dystopia (1.83) + residual,Explained by: GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,Explained by: Generosity,Explained by: Perceptions of corruption,Rank,Cost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index
0,1,Finland,"7,821","7,886","7,756","2,518","1,892","1,258","0,775","0,736","0,109","0,534",41,73.20,25.95,51.05,65.16,77.65,91.02
1,2,Denmark,"7,636","7,710","7,563","2,226","1,953","1,243","0,777","0,719","0,188","0,532",33,84.12,33.23,60.26,68.60,98.75,99.45
2,3,Iceland,"7,557","7,651","7,464","2,320","1,936","1,320","0,803","0,718","0,270","0,191",52,94.86,41.93,70.05,90.22,99.42,77.06
3,4,Switzerland,"7,512","7,586","7,437","2,153","2,026","1,226","0,822","0,677","0,147","0,461",120,123.35,53.54,90.62,128.13,122.09,118.44
4,5,Netherlands,"7,415","7,471","7,359","2,137","1,945","1,206","0,787","0,651","0,271","0,419",86,75.66,36.06,57.10,65.49,76.35,87.99
5,7,Sweden,"7,384","7,454","7,315","2,003","1,920","1,204","0,803","0,724","0,218","0,512",119,71.74,25.86,50.23,65.78,72.61,98.14
6,8,Norway,"7,365","7,440","7,290","1,925","1,997","1,239","0,786","0,728","0,217","0,474",91,100.90,34.68,69.86,97.31,105.49,83.11
7,9,Israel,"7,364","7,426","7,301","2,634","1,826","1,221","0,818","0,568","0,155","0,143",58,88.05,33.94,62.69,76.72,95.31,75.58
8,10,New Zealand,"7,200","7,279","7,120","1,954","1,852","1,235","0,752","0,680","0,245","0,483",87,74.52,34.44,55.73,72.04,70.05,83.63
9,11,Austria,"7,163","7,237","7,089","2,148","1,931","1,165","0,774","0,623","0,193","0,329",7,71.04,27.13,50.46,65.88,66.03,77.25


In [59]:
capCoord_df = pd.read_csv('./Resources/country-capitals.csv', on_bad_lines='skip')
capCoord_df

,CountryName,CapitalName,CapitalLatitude,CapitalLongitude,CountryCode,ContinentName
0,Somaliland,Hargeisa,9.550000,44.050000,NaN,Africa
1,South Georgia and South Sandwich Islands,King Edward Point,-54.283333,-36.500000,GS,Antarctica
2,French Southern and Antarctic Lands,Port-aux-Français,-49.350000,70.216667,TF,Antarctica
3,Palestine,Jerusalem,31.766667,35.233333,PS,Asia
4,Aland Islands,Mariehamn,60.116667,19.900000,AX,Europe
...,...,...,...,...,...,...
238,Northern Cyprus,North Nicosia,35.183333,33.366667,NaN,Europe
239,Hong Kong,NaN,0.000000,0.000000,HK,Asia
240,Heard Island and McDonald Islands,NaN,0.000000,0.000000,HM,Antarctica
241,British Indian Ocean Territory,Diego Garcia,-7.300000,72.400000,IO,Africa


In [61]:
mHapCostCoord_df = mHapCost_df.merge(capCoord_df, how='inner', left_on='Country', right_on='CountryName')
mHapCostCoord_df

,RANK,Country,Happiness score,Whisker-high,Whisker-low,Dystopia (1.83) + residual,Explained by: GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,...,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index,CountryName,CapitalName,CapitalLatitude,CapitalLongitude,CountryCode,ContinentName
0,1,Finland,"7,821","7,886","7,756","2,518","1,892","1,258","0,775","0,736",...,51.05,65.16,77.65,91.02,Finland,Helsinki,60.166667,24.933333,FI,Europe
1,2,Denmark,"7,636","7,710","7,563","2,226","1,953","1,243","0,777","0,719",...,60.26,68.60,98.75,99.45,Denmark,Copenhagen,55.666667,12.583333,DK,Europe
2,3,Iceland,"7,557","7,651","7,464","2,320","1,936","1,320","0,803","0,718",...,70.05,90.22,99.42,77.06,Iceland,Reykjavik,64.150000,-21.950000,IS,Europe
3,4,Switzerland,"7,512","7,586","7,437","2,153","2,026","1,226","0,822","0,677",...,90.62,128.13,122.09,118.44,Switzerland,Bern,46.916667,7.466667,CH,Europe
4,5,Netherlands,"7,415","7,471","7,359","2,137","1,945","1,206","0,787","0,651",...,57.10,65.49,76.35,87.99,Netherlands,Amsterdam,52.350000,4.916667,NL,Europe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,137,Zambia,"3,760","3,902","3,618","1,135","0,930","0,577","0,306","0,525",...,21.90,31.53,22.46,40.33,Zambia,Lusaka,-15.416667,28.283333,ZM,Africa
99,139,Tanzania,"3,702","3,847","3,558","0,735","0,848","0,597","0,425","0,578",...,23.23,30.04,24.86,15.97,Tanzania,Dar es Salaam,-6.800000,39.283333,TZ,Africa
100,144,Zimbabwe,"2,995","3,110","2,880","0,548","0,947","0,690","0,270","0,329",...,28.37,39.88,38.13,19.07,Zimbabwe,Harare,-17.816667,31.033333,ZW,Africa
101,145,Lebanon,"2,955","3,049","2,862","0,216","1,392","0,498","0,631","0,103",...,48.61,66.83,54.25,35.12,Lebanon,Beirut,33.866667,35.500000,LB,Asia


In [75]:
print(mHapCostCoord_df.dtypes)
mHapCostCoord_df['Happiness'] = mHapCostCoord_df['Happiness'].astype('float')
mHapCostCoord_df.dtypes


RANK                                            int64
Country                                        object
Happiness score                                object
Whisker-high                                   object
Whisker-low                                    object
Dystopia (1.83) + residual                     object
Explained by: GDP per capita                   object
Explained by: Social support                   object
Explained by: Healthy life expectancy          object
Explained by: Freedom to make life choices     object
Explained by: Generosity                       object
Explained by: Perceptions of corruption        object
Rank                                            int64
Cost of Living Index                          float64
Rent Index                                    float64
Cost of Living Plus Rent Index                float64
Groceries Index                               float64
Restaurant Price Index                        float64
Local Purchasing Power Index

KeyError: 'Happiness'

In [67]:
%%capture --no-display

happiestplot = mHapCostCoord_df.hvplot.points(
    "CapitalLatitude",
    "CapitalLongitude",
    geo = True,
    tiles = "OSM",
    color = "Country",
    size = 'Happiness score',
    frame_width = 700,
    frame_height = 600,
    alpha = 0.5,
    xlabel = "Longitude (Degrees)",
    ylabel = "Latitude (Degrees)"
)
happiestplot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [CapitalLatitude,CapitalLongitude]   (Country,Happiness score)

In [57]:
bottom20Hap = mHapCost_df.sort_values('Happiness score').head(20)
bottom20Hap

,RANK,Country,Happiness score,Whisker-high,Whisker-low,Dystopia (1.83) + residual,Explained by: GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,Explained by: Generosity,Explained by: Perceptions of corruption,Rank,Cost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index
105,146,Afghanistan,"2,404","2,469","2,339","1,263","0,758","0,000","0,289","0,000","0,089","0,005",1,20.37,2.72,12.09,14.92,12.41,23.04
104,145,Lebanon,"2,955","3,049","2,862","0,216","1,392","0,498","0,631","0,103","0,082","0,034",70,69.62,24.81,48.61,66.83,54.25,35.12
103,144,Zimbabwe,"2,995","3,110","2,880","0,548","0,947","0,690","0,270","0,329","0,106","0,105",139,45.68,8.75,28.37,39.88,38.13,19.07
102,139,Tanzania,"3,702","3,847","3,558","0,735","0,848","0,597","0,425","0,578","0,248","0,270",123,32.81,12.36,23.23,30.04,24.86,15.97
101,137,Zambia,"3,760","3,902","3,618","1,135","0,930","0,577","0,306","0,525","0,203","0,083",138,33.04,9.26,21.90,31.53,22.46,40.33
100,136,India,"3,777","3,828","3,726","0,795","1,167","0,376","0,471","0,647","0,198","0,123",53,24.43,5.30,15.47,26.43,17.56,49.72
99,134,Jordan,"4,152","4,276","4,029","0,695","1,324","0,724","0,675","0,476","0,058","0,200",64,49.19,9.23,30.46,40.81,41.16,31.34
98,131,Ethiopia,"4,241","4,365","4,118","1,375","0,788","0,809","0,457","0,472","0,205","0,136",39,47.49,16.88,33.14,36.68,18.03,10.15
97,129,Egypt,"4,288","4,377","4,198","0,856","1,388","0,732","0,548","0,469","0,041","0,254",36,29.52,5.82,18.41,27.42,23.42,21.54
96,127,Sri Lanka,"4,362","4,487","4,237","0,595","1,415","0,934","0,660","0,529","0,150","0,079",117,31.29,8.23,20.48,37.67,17.27,19.27


In [47]:
crime_df = pd.read_csv('./Resources/WorldCrimeIndex.csv')

crime_df.head()

,Rank,City,Crime Index,Safety Index
0,1,"Caracas, Venezuela",83.98,16.02
1,2,"Pretoria, South Africa",81.98,18.02
2,3,"Celaya, Mexico",81.80,18.20
3,4,"San Pedro Sula, Honduras",80.87,19.13
4,5,"Port Moresby, Papua New Guinea",80.71,19.29


In [48]:
df = crime_df['City'].str.split(",", expand=True)
# crime_df.rename(columns={'City':"Country"})
df.head()

,0,1,2
0,Caracas,Venezuela,None
1,Pretoria,South Africa,None
2,Celaya,Mexico,None
3,San Pedro Sula,Honduras,None
4,Port Moresby,Papua New Guinea,None


In [49]:
crime_df['City'] = df[1]
crime_df.head()

,Rank,City,Crime Index,Safety Index
0,1,Venezuela,83.98,16.02
1,2,South Africa,81.98,18.02
2,3,Mexico,81.80,18.20
3,4,Honduras,80.87,19.13
4,5,Papua New Guinea,80.71,19.29


In [54]:

crime_df.columns=['Crime Rank', 'Country', 'Crime Index', 'Safety Index']
print(crime_df.count())
sr_df = crime_df.sort_values('Country')
print(sr_df.head(20))
crime_df.head()

Crime Rank      453
Country         453
Crime Index     453
Safety Index    453
dtype: int64
     Crime Rank       Country  Crime Index  Safety Index
54           55            AB        63.89         36.11
67           68            AK        62.27         37.73
165         166            AZ        50.29         49.71
150         151            AZ        52.23         47.77
7             8   Afghanistan        79.39         20.61
229         230       Albania        44.50         55.50
129         130       Algeria        53.88         46.12
39           40        Angola        67.45         32.55
76           77     Argentina        60.38         39.62
16           17     Argentina        75.11         24.89
60           61     Argentina        63.33         36.67
427         428       Armenia        21.66         78.34
105         106     Australia        55.76         44.24
75           76     Australia        60.57         39.43
318         319     Australia        35.05         6

,Crime Rank,Country,Crime Index,Safety Index
0,1,Venezuela,83.98,16.02
1,2,South Africa,81.98,18.02
2,3,Mexico,81.80,18.20
3,4,Honduras,80.87,19.13
4,5,Papua New Guinea,80.71,19.29


In [51]:
mHapCosCrim = mHapCost_df.merge(crime_df, how='inner', on='Country')
mHapCosCrim.head()

,RANK,Country,Happiness score,Whisker-high,Whisker-low,Dystopia (1.83) + residual,Explained by: GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,...,Rank,Cost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index,Crime Rank,Crime Index,Safety Index
